In [7]:
#import libraries needed for this proces
import requests
from datetime import datetime
import numpy
import time
import json
from pathlib import Path
import pandas as pd
from os.path import exists

In [62]:
##this block creates functions that can be used to extract the data from the steam API
##maybe we can add this block in the source file as a functions file

def get_request(url, parameters=None):
    #function to create an API request based on URL and  parameters
    print("Your request is being processed.")
    response = requests.get(url=url, params=parameters)
    if response:
        return response.json()
        
    else:
        print("Request unsuccessful, retrying in 5 seconds.")
        time.sleep(5)
        return get_request(url, parameters)
    time.sleep(1)


def getAppList():
    #using the function to get all data id's which include appid and app name 
    url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
    parameters = {"request": all}
    return get_request(url, parameters=parameters)
    print("Your request has been processed successfully, call upon 'data_ids' to preview the full list of IDs and names of all currently available products on Steam as of ", datetime.now().strftime("%d/%m/%Y %H:%M:%S"), ".")
    

def getAppDetails(id):
    #function to collect the raw data from the api per app_id
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": {id}}
    raw_data = get_request(url, parameters=parameters)
    response = raw_data[str(id)]
    if(response['success']):
        data_json = response["data"]
        data_json['collection_details'] = {'created_by': 'our_scraper',
                                            'created_at': datetime.now().strftime("%d/%m/%Y %H:%M:%S")}
        f = open('raw_data.json','a',encoding='utf-8')
        f.write(json.dumps(data_json)+'\n')
        f.close()
        return data_json
    return 
 

In [3]:
#collect all app id's from steam 
data_ids = getAppList()

#Code to collect all app id's in one list and write to a json file called app_ids 
all_app_ids = []
print("Creating a list of all IDs currently available on Steam")

for item in data_ids['applist']['apps']:
    all_app_ids.append(item['appid'])

f = open('app_ids.json','w',encoding='utf-8')
f.write(json.dumps(all_app_ids)+'\n')
f.close()

print("List created sucessfully, the total number of IDs available on Steam is:", len(all_app_ids),".")


Your request is being processed.
Creating a list of all IDs currently available on Steam
List created sucessfully, the total number of IDs available on Steam is: 138564 .


In [ ]:
# this is the file that ultimatly creates the raw data file, and includes the parsing of the nested json categories
counter = 0
limit = 50
keys = ['categories','package_groups','genres','screenshots','movies', 'achievements.highlighted']
Allkeys = ['ids', 'raw', 'categories','package_groups','genres','screenshots','movies', 'achievements.highlighted']
dataframes = {}

## collect app_ids from the all app_ids file created before
r = open('app_ids.json','r',encoding='utf-8')
app_ids = json.load(r)


##creates dataframes for the seperate parsing keys 
for k in Allkeys:
    file = f"dataframe_{k}.csv"
    try:
        dataframes[k] = pd.read_csv(file)
    except:
        print(f"File doesn't exist {file}")
        dataframes[k] = pd.DataFrame()
        pass

##collects data from the api and parses it in different dataframes with the name of the key
for i in app_ids:
    id = str(i)
    
    # if(not hasValueInDF(dataframes['ids'], 'id', i)): 
    if(True):
        counter = counter + 1
        if(counter > limit):
            break
        app_data = getAppDetails(i)
        
        print("Processing data pertaining to Steam ID:"+id)
            
        if(app_data):
            print(f"Parsing id {i} with success")
            steam_appid = str(i)
            result = pd.json_normalize(app_data)
            dataframes['raw'] = pd.concat([dataframes['raw'], result])
            dataframes['ids'] = pd.concat([dataframes['ids'], pd.json_normalize({'id' : i})])

            for key in keys:
                if( key not in dataframes):
                    dataframes[key] = pd.DataFrame()
                try:
                    print(f"Parsing id {i} for key {key}")
                    data = pd.json_normalize(app_data,record_path = [key], meta = ["steam_appid"], errors="ignore")
                    dataframes[key] = pd.concat([dataframes[key],data])            
                    print(key)
                except:
                    pass
    else:
        print(f"Skipping id {str(i)}")


##writes the dataframes to csv files based on the key  
for key in Allkeys:
    print(f"Writing to dataframe {key}")
    file = f"dataframe_{key}.csv"
    print(dataframes[key])
    dataframes[key].to_csv(file)

In [ ]:
#when a dataset was created earlier it is possible add App_ids to your dataset with the code below. 

## collect app_ids from the  app_ids file created before 
r = open('app_ids.json','r',encoding='utf-8')
app_ids = json.load(r)

# check if the json from earlier was created and set limit 
raw_exists = exists("raw_data.json")
counter = 0
limit = 50

#create list of already collected id's 
if (raw_exists):
    #create list of collected id's 
    col_ids = []
    with open('raw_data.json') as f: 
        for jsonObj in f: 
            rawDict = json.loads(jsonObj)
            col_ids.append(rawDict["steam_appid"])

## getting the new data
for i in app_ids: 
    if i not in col_ids:
        if(counter > limit):
            break
        id = str(i)
        counter = counter + 1
        app_data = getAppDetails(i)
        print("Processing data pertaining to Steam ID:"+id)
        if(app_data):
            print(f"Parsing id {i} with success")
            steam_appid = str(i)
            result = pd.json_normalize(app_data)
            dataframes['raw'] = pd.concat([dataframes['raw'], result])
            dataframes['ids'] = pd.concat([dataframes['ids'], pd.json_normalize({'id' : i})])

            for key in keys:
                if( key not in dataframes):
                    dataframes[key] = pd.DataFrame()
                try:
                    print(f"Parsing id {i} for key {key}")
                    data = pd.json_normalize(app_data,record_path = [key], meta = ["steam_appid"], errors="ignore")
                    dataframes[key] = pd.concat([dataframes[key],data])            
                    print(key)
                except:
                    pass   
    else:
        print(f"Skipping id {str(i)}")
        

for key in Allkeys:
    print(f"Writing to dataframe {key}")
    file = f"dataframe_{key}.csv"
    print(dataframes[key])
    dataframes[key].to_csv(file)
